In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#The data has been cleaned for us already, so no concerns about that. Start by grabbing previous results, seeds, and MasseyOrdinals
df_seeds = pd.read_csv('../input/ncaam-march-mania-2021/MNCAATourneySeeds.csv')
df_tour = pd.read_csv('../input/ncaam-march-mania-2021/MRegularSeasonCompactResults.csv')
df_massey = pd.read_csv('../input/ncaam-march-mania-2021/MMasseyOrdinals.csv')

In [ ]:
df_seeds.isnull().sum()

In [ ]:
df_tour.isnull().sum()

In [ ]:
df_massey.isnull().sum()

In [ ]:
#Convert the seed values to an integer, we are not concerned about the conference
def seedToInt(seed):
    #Since each seed begins with a letter (W01), skip the first element
    return int(seed[1:3])

#Apply the function  , Seed를 int_seed로 Seed는 삭제 + drop()과 inplace =True를 사용함으로 데이터프레임 값에 리턴값을 설정하지 않아도 된다. 
df_seeds['int_seed'] = df_seeds['Seed'].apply(seedToInt)
df_seeds.drop('Seed', inplace=True, axis=1)
df_seeds.head()

In [ ]:
#Only use the final massey ranking for the year
df_finalmassey = df_massey[df_massey['RankingDayNum']==max(df_massey['RankingDayNum'].values)]

#Need to group them by team and season, and average all the rankings from each system
df_gb = df_finalmassey.groupby(['TeamID', 'Season'])['OrdinalRank'].mean().unstack('TeamID')

#Reset the index to simplify grabbing the values. Not particularly clean, but it works.
df_SeasonTeamID = df_gb.reset_index()



In [ ]:
df_check = df_SeasonTeamID.isnull().values.any()
print(df_check)

In [ ]:
df_SeasonTeamID

In [ ]:
df_SeasonTeamID.isnull().sum()

In [ ]:
df_SeasonTeamID.dropna(axis=1)


In [ ]:
df_SeasonTeamID_2 = df_SeasonTeamID.dropna(axis=1)


In [ ]:
df_SeasonTeamID_2.isnull().sum()

In [ ]:
#Get all of the seasons we now have rankings for
massey_seasons = list(df_SeasonTeamID_2['Season'].values)

In [ ]:
df_tour

In [ ]:
df_tour.isnull().sum()

In [ ]:
#Don't care about the Day, score, location or OT right now
df_tour.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_tour.tail()

In [ ]:
#Subset the tour data using the first year in our massey data as the starting point
df_subs_tour = df_tour[df_tour['Season'] >= min(massey_seasons)]

In [ ]:
df_subs_tour

In [ ]:
#Fix the index and check how our data looks
df_subs_tour.reset_index(inplace=True, drop=True)
df_subs_tour

In [ ]:
#Now match up the Massey Rankings for each winning and losing team
WMassey = [0]*len(df_subs_tour)
LMassey = [0]*len(df_subs_tour)

In [ ]:
#Now match up the Massey Rankings for each winning and losing team
WMassey = [0]*len(df_subs_tour)
LMassey = [0]*len(df_subs_tour)

for ind, row in df_subs_tour.iterrows(): 
    season = row['Season']
    wid = row['WTeamID']
    lid = row['LTeamID']
    WMassey[ind] = df_SeasonTeamID[df_SeasonTeamID['Season']==season][wid].values[0]
    LMassey[ind] = df_SeasonTeamID[df_SeasonTeamID['Season']==season][lid].values[0]




In [ ]:
#Use the lists to create our new columns
df_subs_tour['WMassey'] = WMassey
df_subs_tour['LMassey'] = LMassey

df_subs_tour.head(10)